# CategoryVector 演示

这个笔记本演示了 CategoryVector 项目的核心功能，包括：
1. 加载类别数据
2. 构建向量索引
3. 执行搜索查询

通过这个笔记本，您可以直观地了解整个工作流程，并查看详细的中间结果。

## 1. 环境设置

首先导入必要的模块并设置日志级别

In [ ]:
import os
import sys
import json
import time
import numpy as np
from pathlib import Path
from IPython.display import display, HTML
import pandas as pd
import matplotlib.pyplot as plt

# 确保可以导入项目模块
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# 导入项目模块
from src.categoryvector.config import CategoryVectorConfig
from src.categoryvector.data_processing import CategoryProcessor
from src.categoryvector.vector_generation import VectorGenerator
from src.categoryvector.vector_storage import VectorStorage
from src.categoryvector.utils.logging_utils import setup_logger
from src.categoryvector.models import Category

# 设置日志
logger = setup_logger("categoryvector_demo", level="INFO")

## 2. 配置参数

设置项目参数，包括数据路径、模型名称等

In [ ]:
# 数据和输出路径
DATA_DIR = project_root / "data"
CATEGORIES_FILE = DATA_DIR / "categories.json"
OUTPUT_DIR = DATA_DIR / "vectors"

# 如果没有示例数据，使用示例数据
if not CATEGORIES_FILE.exists():
    CATEGORIES_FILE = DATA_DIR / "sample_categories.json"
    print(f"使用示例数据: {CATEGORIES_FILE}")

# 确保输出目录存在
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 模型参数
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
VECTOR_DIM = 384

print(f"类别数据文件: {CATEGORIES_FILE}")
print(f"输出目录: {OUTPUT_DIR}")
print(f"模型名称: {MODEL_NAME}")
print(f"向量维度: {VECTOR_DIM}")

## 3. 加载类别数据

从JSON文件中加载商品分类数据

In [ ]:
# 创建配置
config = CategoryVectorConfig(
    model_name=MODEL_NAME,
    data_dir=DATA_DIR,
    vector_dim=VECTOR_DIM,
    log_level="INFO"
)

# 加载类别数据
print(f"开始加载类别数据...")
start_time = time.time()
processor = CategoryProcessor(config)
processor.load_from_json(CATEGORIES_FILE)
load_time = time.time() - start_time

print(f"加载完成，共 {len(processor.categories)} 个类别，耗时 {load_time:.2f} 秒")

# 显示类别样例
sample_size = min(5, len(processor.categories))
sample_categories = list(processor.categories.values())[:sample_size]

sample_data = []
for cat in sample_categories:
    sample_data.append({
        "ID": cat.id,
        "路径": cat.path,
        "层级": cat.level_depth,
        "描述": cat.description[:50] + '...' if cat.description and len(cat.description) > 50 else cat.description
    })

display(pd.DataFrame(sample_data))

## 4. 构建向量索引

生成类别向量并构建FAISS索引

In [ ]:
# 4.1 初始化向量生成器
print(f"开始加载模型 {MODEL_NAME}...")
start_time = time.time()
generator = VectorGenerator(model_name=MODEL_NAME, config=config)
model_load_time = time.time() - start_time
print(f"模型加载完成，耗时 {model_load_time:.2f} 秒")

# 4.2 为类别生成向量
print(f"\n开始为 {len(processor.categories)} 个类别生成向量...")
vector_start_time = time.time()
processed_count = 0

# 进度条更新函数
def update_progress(current, total, bar_length=50):
    progress = min(1.0, current/total)
    arrow = '=' * int(round(progress * bar_length))
    spaces = ' ' * (bar_length - len(arrow))
    sys.stdout.write(f"\r进度: [{arrow + spaces}] {int(progress * 100)}% ({current}/{total})")
    sys.stdout.flush()

for cat_id, category in processor.categories.items():
    category = generator.enrich_category_vectors(category)
    processed_count += 1
    if processed_count % 5 == 0 or processed_count == len(processor.categories):
        update_progress(processed_count, len(processor.categories))

vector_time = time.time() - vector_start_time
print(f"\n向量生成完成，共处理 {processed_count} 个类别，耗时 {vector_time:.2f} 秒")

# 4.3 构建索引
print(f"\n开始构建FAISS索引...")
index_start_time = time.time()
storage = VectorStorage(
    dimension=generator.model.get_sentence_embedding_dimension(),
    config=config
)

# 添加类别到索引
for cat_id, category in processor.categories.items():
    storage.add_category(category)
    
index_time = time.time() - index_start_time
print(f"索引构建完成，包含 {storage.index.ntotal} 个向量，耗时 {index_time:.2f} 秒")

# 4.4 保存索引
print(f"\n开始保存索引到 {OUTPUT_DIR}...")
save_start_time = time.time()
storage.save(OUTPUT_DIR)
save_time = time.time() - save_start_time

# 获取文件大小
index_file_size = (OUTPUT_DIR / "index.faiss").stat().st_size / (1024 * 1024)
categories_file_size = (OUTPUT_DIR / "categories.json").stat().st_size / (1024 * 1024)

print(f"索引保存完成，耗时 {save_time:.2f} 秒")
print(f"FAISS索引文件大小: {index_file_size:.2f} MB")
print(f"类别数据文件大小: {categories_file_size:.2f} MB")

# 4.5 显示总体构建统计信息
total_build_time = model_load_time + vector_time + index_time + save_time
print(f"\n索引构建总耗时: {total_build_time:.2f} 秒")

# 构建过程时间分布图
times = [
    ('模型加载', model_load_time),
    ('向量生成', vector_time),
    ('索引构建', index_time),
    ('索引保存', save_time)
]

plt.figure(figsize=(10, 6))
plt.bar([t[0] for t in times], [t[1] for t in times], color='skyblue')
plt.title('构建过程耗时分布 (秒)')
plt.xlabel('步骤')
plt.ylabel('耗时 (秒)')
for i, (_, t) in enumerate(times):
    plt.text(i, t + 0.1, f"{t:.2f}s", ha='center')
plt.tight_layout()
plt.show()

## 5. 执行搜索查询

使用构建好的索引执行各种搜索查询

In [ ]:
# 5.1 准备搜索环境
def perform_search(query, level=None, top_k=5, threshold=0.5, exclusions=None):
    """执行搜索并格式化显示结果"""
    print(f"\n搜索查询: '{query}'")
    if level:
        print(f"层级限制: {level}")
    if exclusions:
        print(f"排除词: {', '.join(exclusions)}")
    print(f"参数: top_k={top_k}, threshold={threshold}")
    print("-" * 80)
    
    # 生成查询向量
    query_vector = generator.generate_query_vector(query)
    
    # 处理排除词
    exclusion_vector = None
    if exclusions:
        exclusion_vector = generator.generate_exclusion_vector(exclusions)
    
    # 执行搜索
    start_time = time.time()
    if level:
        # 这里假设 search_by_level 方法已经支持 exclusion_vector
        # 如果不支持，您需要修改搜索逻辑
        if exclusion_vector is not None:
            # 调整查询向量
            query_vector = query_vector - 0.2 * exclusion_vector
        results = storage.search_by_level(query_vector, level=level, top_k=top_k, threshold=threshold)
    else:
        if exclusion_vector is not None:
            # 调整查询向量
            query_vector = query_vector - 0.2 * exclusion_vector
        results = storage.search(query_vector, top_k=top_k, threshold=threshold)
    
    search_time = time.time() - start_time
    
    # 显示结果
    print(f"找到 {len(results)} 个结果 (耗时 {search_time:.4f} 秒)")
    
    if results:
        result_data = []
        for i, (category, score) in enumerate(results):
            result_data.append({
                "排名": i+1,
                "ID": category.id,
                "相似度": f"{score:.4f}",
                "路径": category.path,
                "描述": category.description[:50] + '...' if category.description and len(category.description) > 50 else category.description
            })
        
        return pd.DataFrame(result_data)
    else:
        print("没有找到匹配的结果")
        return None

### 5.2 基本搜索示例

In [ ]:
# 基本搜索 - 电子产品
results_df = perform_search(query="电子产品", top_k=5, threshold=0.3)
if results_df is not None:
    display(results_df)

### 5.3 层级搜索示例

In [ ]:
# 按层级搜索 - 找第二层级的手机相关分类
results_df = perform_search(query="手机", level=2, top_k=3, threshold=0.3)
if results_df is not None:
    display(results_df)

### 5.4 使用排除词的搜索示例

In [ ]:
# 带排除词的搜索 - 电脑，但排除笔记本
results_df = perform_search(query="电脑", exclusions=["笔记本"], top_k=5, threshold=0.3)
if results_df is not None:
    display(results_df)

### 5.5 多语言搜索示例

In [ ]:
# 多语言搜索 - 使用英文查询
results_df = perform_search(query="Mobile Phone", top_k=3, threshold=0.3)
if results_df is not None:
    display(results_df)

### 5.6 自定义查询示例

您可以在下面尝试自己的查询

In [ ]:
# 自定义查询
custom_query = "智能手机" # 修改为您想要的查询
custom_level = None      # 设置为数字以限制层级，如1、2、3
custom_exclusions = []   # 添加排除词，如 ["配件", "充电器"]

results_df = perform_search(
    query=custom_query, 
    level=custom_level, 
    exclusions=custom_exclusions if custom_exclusions else None,
    top_k=5, 
    threshold=0.3
)
if results_df is not None:
    display(results_df)

## 6. 结果分析与可视化

In [ ]:
# 6.1 执行多个查询并比较结果
print("执行多个查询并比较结果...")
queries = ["手机", "智能手机", "手机配件", "电脑"]
query_results = {}

for query in queries:
    query_vector = generator.generate_query_vector(query)
    results = storage.search(query_vector, top_k=3, threshold=0.2)
    query_results[query] = [(cat.path, score) for cat, score in results]

# 创建对比表格
comparison_data = []
for query, results in query_results.items():
    for i, (path, score) in enumerate(results):
        comparison_data.append({
            "查询": query,
            "排名": i+1,
            "分类路径": path,
            "相似度": score
        })

# 显示对比表格
comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)

# 6.2 查询相似度可视化
plt.figure(figsize=(12, 6))
for query in queries:
    scores = [r[1] for r in query_results[query]]
    plt.plot(range(1, len(scores)+1), scores, marker='o', label=query)

plt.title('不同查询的结果相似度比较')
plt.xlabel('结果排名')
plt.ylabel('相似度')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.ylim(0, 1.0)
plt.show()

## 7. 总结与结论

在这个笔记本中，我们完成了以下任务：

1. 加载了类别数据
2. 使用预训练模型生成了类别向量
3. 构建并保存了FAISS向量索引
4. 执行了多种不同类型的搜索查询
5. 分析和可视化了搜索结果

通过这个演示，我们可以看到基于向量的商品分类搜索系统如何工作，以及如何通过不同的参数和排除词来调整搜索结果。这种方法特别适合处理模糊匹配和跨语言搜索的场景。